# 2주차 이상처리에 적합한 가상 데이터를 활용한 데이터 클리닝

### 1. 문제 풀이 준비

먼저, 다양한 전처리 기법을 적용할 수 있는 가상의 데이터를 제공하겠습니다. 

데이터는 Pandas DataFrame 형식으로 제공하며, 각 열은 다음과 같은 속성을 가집니다:

- **TransactionID**: 거래 고유 ID
- **CustomerID**: 고객 고유 ID
- **PurchaseAmount**: 구매 금액 (USD)
- **PurchaseDate**: 구매 날짜
- **ProductCategory**: 제품 카테고리 (범주형 데이터)
- **CustomerAge**: 고객 나이
- **CustomerGender**: 고객 성별 (범주형 데이터)
- **ReviewScore**: 제품 리뷰 점수 (1~5 사이의 값, 결측값 포함)

가상의 데이터를 생성하는 파이썬 코드는 다음과 같습니다:

In [57]:
import pandas as pd
import numpy as np

# 가상 데이터 생성
data = {
    'TransactionID': range(1, 21),
    'CustomerID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'PurchaseAmount': [250, -50, 3000000, 450, 0, 300, 200, 150, -10, 800, 50, 75, 400, np.nan, 600, 1000, 20, 5000, 150, 80],
    'PurchaseDate': pd.date_range(start='2024-01-01', periods=20, freq='ME').tolist(),
    'ProductCategory': ['Electronics', 'Clothing', 'Electronics', 'Home', 'Electronics', 'Home', 'Clothing', 'Home', 'Clothing', 'Electronics', 'Electronics', 'Home', 'Clothing', 'Electronics', 'Home', 'Home', 'Clothing', 'Electronics', 'Home', 'Electronics'],
    'CustomerAge': [25, 35, 45, np.nan, 22, 29, 33, 41, 27, 36, 28, 34, 42, 39, 24, 30, 32, 40, 38, 26],
    'CustomerGender': ['Male', 'Female', 'Female', 'Male', 'Female', 'Male', 'Female', np.nan, 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female'],
    'ReviewScore': [5, np.nan, 4, 3, 2, 5, 3, 4, 1, 2, np.nan, 4, 5, 3, 4, np.nan, 1, 5, 2, 4]
}

df = pd.DataFrame(data)


### 2. 요구 사항

다음은 데이터 전처리를 위한 문제입니다:

1. 결측값 처리:
    - `PurchaseAmount`, `CustomerAge`, `CustomerGender`, `ReviewScore` 열의 결측값을 적절히 처리하세요.
2. 이상치 처리:
    - `PurchaseAmount` 열에서 비정상적으로 큰 값과 음수 값을 처리하세요.
3. 중복 데이터 제거:
    - 중복된 `TransactionID`가 있는 경우 제거하세요.
4. 데이터 타입 변환:
    - `PurchaseDate` 열의 데이터 타입을 날짜 형식으로 변환하세요.
5. 정규화:
    - `PurchaseAmount` 열을 정규화하세요.
6. 범주형 데이터 선별 및 인코딩:
    - `ProductCategory`와 `CustomerGender` 열을 인코딩하세요.
7. 샘플링:
    - 데이터를 무작위로 5개 샘플링하세요.

### **숙제 정보**

■ 난이도 : 🟡중

■ 실습 범위 : 2주차

■ 사용 언어 및 라이브러리 : pandas, numpy

---

In [58]:
#데이터 출력
df

TransactionID  CustomerID  PurchaseAmount PurchaseDate ProductCategory  \
0               1         101           250.0   2024-01-31     Electronics   
1               2         102           -50.0   2024-02-29        Clothing   
2               3         103       3000000.0   2024-03-31     Electronics   
3               4         104           450.0   2024-04-30            Home   
4               5         105             0.0   2024-05-31     Electronics   
5               6         106           300.0   2024-06-30            Home   
6               7         107           200.0   2024-07-31        Clothing   
7               8         108           150.0   2024-08-31            Home   
8               9         109           -10.0   2024-09-30        Clothing   
9              10         110           800.0   2024-10-31     Electronics   
10             11         101            50.0   2024-11-30     Electronics   
11             12         102            75.0   2024-12-31            Home   
12             13         103           400.0   2025-01-31        Clothing   
13             14         104             NaN   2025-02-28     Electronics   
14             15         105           600.0   2025-03-31            Home   
15             16         106          1000.0   2025-04-30            Home   
16             17         107            20.0   2025-05-31        Clothing   
17             18         108          5000.0   2025-06-30     Electronics   
18             19         109           150.0   2025-07-31            Home   
19             20         110            80.0   2025-08-31     Electronics   

    CustomerAge CustomerGender  ReviewScore  
0          25.0           Male          5.0  
1          35.0         Female          NaN  
2          45.0         Female          4.0  
3           NaN           Male          3.0  
4          22.0         Female          2.0  
5          29.0           Male          5.0  
6          33.0         Female          3.0  
7          41.0            NaN          4.0  
8          27.0           Male          1.0  
9          36.0         Female          2.0  
10         28.0           Male          NaN  
11         34.0         Female          4.0  
12         42.0           Male          5.0  
13         39.0         Female          3.0  
14         24.0           Male          4.0  
15         30.0         Female          NaN  
16         32.0           Male          1.0  
17         40.0         Female          5.0  
18         38.0           Male          2.0  
19         26.0         Female          4.0

### 1. 결측값 처리:
PurchaseAmount, CustomerAge, CustomerGender, ReviewScore 열의 결측값을 적절히 처리하세요.

In [59]:
#평균값으로 결측치 처리 - PurchaseAmount

#결측치 확인
df['PurchaseAmount'].isnull()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
17    False
18    False
19    False
Name: PurchaseAmount, dtype: bool

In [60]:
#결측치 처리
df['PurchaseAmount'] = df['PurchaseAmount'].fillna(df['PurchaseAmount'].mean())

#출력
df['PurchaseAmount']

0     2.500000e+02
1    -5.000000e+01
2     3.000000e+06
3     4.500000e+02
4     0.000000e+00
5     3.000000e+02
6     2.000000e+02
7     1.500000e+02
8    -1.000000e+01
9     8.000000e+02
10    5.000000e+01
11    7.500000e+01
12    4.000000e+02
13    1.583929e+05
14    6.000000e+02
15    1.000000e+03
16    2.000000e+01
17    5.000000e+03
18    1.500000e+02
19    8.000000e+01
Name: PurchaseAmount, dtype: float64

In [61]:
#평균값으로 결측치 처리 - CustomerAge

#결측치 확인
df['CustomerAge'].isnull()

0     False
1     False
2     False
3      True
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: CustomerAge, dtype: bool

In [62]:
#결측치 처리
df['CustomerAge'] = df['CustomerAge'].fillna(df['CustomerAge'].mean())

#출력
df['CustomerAge']

0     25.000000
1     35.000000
2     45.000000
3     32.947368
4     22.000000
5     29.000000
6     33.000000
7     41.000000
8     27.000000
9     36.000000
10    28.000000
11    34.000000
12    42.000000
13    39.000000
14    24.000000
15    30.000000
16    32.000000
17    40.000000
18    38.000000
19    26.000000
Name: CustomerAge, dtype: float64

In [63]:
#최빈값으로 결측치 처리 - CustomerGender

#결측치 확인
df['CustomerGender'].isnull()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: CustomerGender, dtype: bool

In [64]:
#결측치 처리
df['CustomerGender'] = df['CustomerGender'].fillna(df['CustomerGender'].mode()[0])

#출력
df['CustomerGender']

0       Male
1     Female
2     Female
3       Male
4     Female
5       Male
6     Female
7     Female
8       Male
9     Female
10      Male
11    Female
12      Male
13    Female
14      Male
15    Female
16      Male
17    Female
18      Male
19    Female
Name: CustomerGender, dtype: object

In [65]:
#평균값으로 결측치 처리 - ReviewScore

#결측치 확인
df['ReviewScore'].isnull()

0     False
1      True
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
11    False
12    False
13    False
14    False
15     True
16    False
17    False
18    False
19    False
Name: ReviewScore, dtype: bool

In [66]:
#결측치 처리
df['ReviewScore'] = df['ReviewScore'].fillna(df['ReviewScore'].mean())

#출력
df['ReviewScore']

0     5.000000
1     3.352941
2     4.000000
3     3.000000
4     2.000000
5     5.000000
6     3.000000
7     4.000000
8     1.000000
9     2.000000
10    3.352941
11    4.000000
12    5.000000
13    3.000000
14    4.000000
15    3.352941
16    1.000000
17    5.000000
18    2.000000
19    4.000000
Name: ReviewScore, dtype: float64

---

### 2. 이상치 처리:
PurchaseAmount 열에서 비정상적으로 큰 값과 음수 값을 처리하세요.

In [67]:
#데이터 출력
df['PurchaseAmount']

0     2.500000e+02
1    -5.000000e+01
2     3.000000e+06
3     4.500000e+02
4     0.000000e+00
5     3.000000e+02
6     2.000000e+02
7     1.500000e+02
8    -1.000000e+01
9     8.000000e+02
10    5.000000e+01
11    7.500000e+01
12    4.000000e+02
13    1.583929e+05
14    6.000000e+02
15    1.000000e+03
16    2.000000e+01
17    5.000000e+03
18    1.500000e+02
19    8.000000e+01
Name: PurchaseAmount, dtype: float64

In [68]:
#이상치 처리
#음수값 제거
df = df[df['PurchaseAmount'] >= 0]

#PurchaseAmount의 비정상적으로 큰 값 제거 (예: 1,000,000 이상)
df = df[df['PurchaseAmount'] < 1000000]

df

TransactionID  CustomerID  PurchaseAmount PurchaseDate ProductCategory  \
0               1         101      250.000000   2024-01-31     Electronics   
3               4         104      450.000000   2024-04-30            Home   
4               5         105        0.000000   2024-05-31     Electronics   
5               6         106      300.000000   2024-06-30            Home   
6               7         107      200.000000   2024-07-31        Clothing   
7               8         108      150.000000   2024-08-31            Home   
9              10         110      800.000000   2024-10-31     Electronics   
10             11         101       50.000000   2024-11-30     Electronics   
11             12         102       75.000000   2024-12-31            Home   
12             13         103      400.000000   2025-01-31        Clothing   
13             14         104   158392.894737   2025-02-28     Electronics   
14             15         105      600.000000   2025-03-31            Home   
15             16         106     1000.000000   2025-04-30            Home   
16             17         107       20.000000   2025-05-31        Clothing   
17             18         108     5000.000000   2025-06-30     Electronics   
18             19         109      150.000000   2025-07-31            Home   
19             20         110       80.000000   2025-08-31     Electronics   

    CustomerAge CustomerGender  ReviewScore  
0     25.000000           Male     5.000000  
3     32.947368           Male     3.000000  
4     22.000000         Female     2.000000  
5     29.000000           Male     5.000000  
6     33.000000         Female     3.000000  
7     41.000000         Female     4.000000  
9     36.000000         Female     2.000000  
10    28.000000           Male     3.352941  
11    34.000000         Female     4.000000  
12    42.000000           Male     5.000000  
13    39.000000         Female     3.000000  
14    24.000000           Male     4.000000  
15    30.000000         Female     3.352941  
16    32.000000           Male     1.000000  
17    40.000000         Female     5.000000  
18    38.000000           Male     2.000000  
19    26.000000         Female     4.000000

---

### 3. 중복 데이터 제거:
중복된 TransactionID가 있는 경우 제거하세요.

In [69]:
#데이터 출력
df['TransactionID']

0      1
3      4
4      5
5      6
6      7
7      8
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
Name: TransactionID, dtype: int64

In [70]:
#중복값 제거
df = df.drop_duplicates(subset = 'TransactionID')

#출력
df

TransactionID  CustomerID  PurchaseAmount PurchaseDate ProductCategory  \
0               1         101      250.000000   2024-01-31     Electronics   
3               4         104      450.000000   2024-04-30            Home   
4               5         105        0.000000   2024-05-31     Electronics   
5               6         106      300.000000   2024-06-30            Home   
6               7         107      200.000000   2024-07-31        Clothing   
7               8         108      150.000000   2024-08-31            Home   
9              10         110      800.000000   2024-10-31     Electronics   
10             11         101       50.000000   2024-11-30     Electronics   
11             12         102       75.000000   2024-12-31            Home   
12             13         103      400.000000   2025-01-31        Clothing   
13             14         104   158392.894737   2025-02-28     Electronics   
14             15         105      600.000000   2025-03-31            Home   
15             16         106     1000.000000   2025-04-30            Home   
16             17         107       20.000000   2025-05-31        Clothing   
17             18         108     5000.000000   2025-06-30     Electronics   
18             19         109      150.000000   2025-07-31            Home   
19             20         110       80.000000   2025-08-31     Electronics   

    CustomerAge CustomerGender  ReviewScore  
0     25.000000           Male     5.000000  
3     32.947368           Male     3.000000  
4     22.000000         Female     2.000000  
5     29.000000           Male     5.000000  
6     33.000000         Female     3.000000  
7     41.000000         Female     4.000000  
9     36.000000         Female     2.000000  
10    28.000000           Male     3.352941  
11    34.000000         Female     4.000000  
12    42.000000           Male     5.000000  
13    39.000000         Female     3.000000  
14    24.000000           Male     4.000000  
15    30.000000         Female     3.352941  
16    32.000000           Male     1.000000  
17    40.000000         Female     5.000000  
18    38.000000           Male     2.000000  
19    26.000000         Female     4.000000

---

### 4. 데이터 타입 변환:
PurchaseDate 열의 데이터 타입을 날짜 형식으로 변환하세요.

In [71]:
#데이터 타입 변환
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

#출력
df['PurchaseDate']

0    2024-01-31
3    2024-04-30
4    2024-05-31
5    2024-06-30
6    2024-07-31
7    2024-08-31
9    2024-10-31
10   2024-11-30
11   2024-12-31
12   2025-01-31
13   2025-02-28
14   2025-03-31
15   2025-04-30
16   2025-05-31
17   2025-06-30
18   2025-07-31
19   2025-08-31
Name: PurchaseDate, dtype: datetime64[ns]

---

### 5. 정규화:
PurchaseAmount 열을 정규화하세요.


In [72]:
#데이터 출력
df['PurchaseAmount']

0        250.000000
3        450.000000
4          0.000000
5        300.000000
6        200.000000
7        150.000000
9        800.000000
10        50.000000
11        75.000000
12       400.000000
13    158392.894737
14       600.000000
15      1000.000000
16        20.000000
17      5000.000000
18       150.000000
19        80.000000
Name: PurchaseAmount, dtype: float64

In [76]:
#정규화 진행
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df['PurchaseAmount'] = scaler.fit_transform(df[['PurchaseAmount']])

In [77]:
#출력
df['PurchaseAmount']

0     0.001578
3     0.002841
4     0.000000
5     0.001894
6     0.001263
7     0.000947
9     0.005051
10    0.000316
11    0.000474
12    0.002525
13    1.000000
14    0.003788
15    0.006313
16    0.000126
17    0.031567
18    0.000947
19    0.000505
Name: PurchaseAmount, dtype: float64

---

### 6. 범주형 데이터 선별 및 인코딩:
ProductCategory와 CustomerGender 열을 인코딩하세요.


In [81]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['ProductCategory'] = encoder.fit_transform(df['ProductCategory'])
df['CustomerGender'] = encoder.fit_transform(df['CustomerGender'])

In [83]:
#츨력 - ProductCategory
df['ProductCategory']

0     1
3     2
4     1
5     2
6     0
7     2
9     1
10    1
11    2
12    0
13    1
14    2
15    2
16    0
17    1
18    2
19    1
Name: ProductCategory, dtype: int64

In [84]:
#출력 - CustomerGender
df['CustomerGender']

0     1
3     1
4     0
5     1
6     0
7     0
9     0
10    1
11    0
12    1
13    0
14    1
15    0
16    1
17    0
18    1
19    0
Name: CustomerGender, dtype: int64

---

### 7. 샘플링:
데이터를 무작위로 5개 샘플링하세요.

In [86]:
#샘플링
sampled_df = df.sample(n=5, random_state=42)

In [87]:
#출력
df

TransactionID  CustomerID  PurchaseAmount PurchaseDate  ProductCategory  \
0               1         101        0.001578   2024-01-31                1   
3               4         104        0.002841   2024-04-30                2   
4               5         105        0.000000   2024-05-31                1   
5               6         106        0.001894   2024-06-30                2   
6               7         107        0.001263   2024-07-31                0   
7               8         108        0.000947   2024-08-31                2   
9              10         110        0.005051   2024-10-31                1   
10             11         101        0.000316   2024-11-30                1   
11             12         102        0.000474   2024-12-31                2   
12             13         103        0.002525   2025-01-31                0   
13             14         104        1.000000   2025-02-28                1   
14             15         105        0.003788   2025-03-31                2   
15             16         106        0.006313   2025-04-30                2   
16             17         107        0.000126   2025-05-31                0   
17             18         108        0.031567   2025-06-30                1   
18             19         109        0.000947   2025-07-31                2   
19             20         110        0.000505   2025-08-31                1   

    CustomerAge  CustomerGender  ReviewScore  
0     25.000000               1     5.000000  
3     32.947368               1     3.000000  
4     22.000000               0     2.000000  
5     29.000000               1     5.000000  
6     33.000000               0     3.000000  
7     41.000000               0     4.000000  
9     36.000000               0     2.000000  
10    28.000000               1     3.352941  
11    34.000000               0     4.000000  
12    42.000000               1     5.000000  
13    39.000000               0     3.000000  
14    24.000000               1     4.000000  
15    30.000000               0     3.352941  
16    32.000000               1     1.000000  
17    40.000000               0     5.000000  
18    38.000000               1     2.000000  
19    26.000000               0     4.000000

In [88]:
#샘플링 데이터 출력
sampled_df

TransactionID  CustomerID  PurchaseAmount PurchaseDate  ProductCategory  \
0               1         101        0.001578   2024-01-31                1   
3               4         104        0.002841   2024-04-30                2   
7               8         108        0.000947   2024-08-31                2   
18             19         109        0.000947   2025-07-31                2   
14             15         105        0.003788   2025-03-31                2   

    CustomerAge  CustomerGender  ReviewScore  
0     25.000000               1          5.0  
3     32.947368               1          3.0  
7     41.000000               0          4.0  
18    38.000000               1          2.0  
14    24.000000               1          4.0